In [2]:
from tkinter import *
from owlready2 import *
import pandas as pd
import datetime, math

In [3]:
class ResearchMentorApp:
    def __init__(self, master):
        frame = Frame(master)
        frame.pack()
        
        # Quit and enter buttons
        self.button = Button(frame, text="QUIT", fg="red", command=frame.quit)
        self.button.pack(side=RIGHT)
        
        self.button = Button(frame, text="QUIT", fg="red", command=frame.quit)
        self.button.pack(side=RIGHT)
        
        # search boxes
        SEARCH_BY_OPTIONS = ["Affliation","MESH"]
        self.search_by_var = StringVar(master)
        self.search_by_var.set(SEARCH_BY_OPTIONS[0])
        option = OptionMenu(master, self.search_by_var,*SEARCH_BY_OPTIONS)
        option.pack()
        
#         self.hi_there = Button(frame, text="Hello", command=self.say_hi)
#         self.hi_there.pack(side=LEFT)
#     def say_hi(self):
#         print("hi there, everyone!")

    def load_ontology():
        onto = get_ontology('main-ResearchMentorOntology.owl')
        vivoNS = onto.get_namespace("http://vivoweb.org/ontology/core")
        meshNS = onto.get_namespace("http://phenomebrowser.net/ontologies/mesh/mesh.owl#")
        onto.load()
        return onto, vivoNS

root = Tk()
app = ResearchMentorApp(root)
root.mainloop()
root.destroy()

In [5]:
class App:
    def __init__(self, master):
        frame = Frame(master)
        frame.pack()
        
        self.button = Button(frame, text="QUIT", fg="red", command=frame.quit)
        self.button.pack(side=LEFT)
        self.hi_there = Button(frame, text="Hellow", command=self.say_hi)
        self.hi_there.pack(side=LEFT)
    def say_hi(self):
        print("hi there, everyone!")

root = Tk()
app = App(root)
root.mainloop()
root.destroy()

hi there, everyone!


In [3]:
# hello world app
# root = Tk()
# w = Label(root, text="Hello, world!")
# w.pack()
# root.mainloop()